<figure>
<img src="../Imagenes/logo-final-ap.png"  width="80" height="80" align="left"/> 
</figure>

# <span style="color:blue"><left>Aprendizaje Profundo</left></span>

# <span style="color:red"><center>Gradientes de la Política</center></span>

<center> Policy gradients</center>

##   <span style="color:blue">Autores</span>

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 

##   <span style="color:blue">Asesora Medios y Marketing digital</span>
 

4. Maria del Pilar Montenegro, pmontenegro88@gmail.com 

## <span style="color:blue">Asistentes</span>

## <span style="color:blue">Referencias</span>

1. [Alvaro Montenegro y Daniel Montenegro, Inteligencia Artificial y Aprendizaje Profundo, 2021](https://github.com/AprendizajeProfundo/Diplomado)
1. [Maxim Lapan, Deep Reinforcement Learning Hands-On: Apply modern RL methods to practical problems of chatbots, robotics, discrete optimization, web automation, and more, 2nd Edition, 2020](http://library.lol/main/F4D1A90C476A576238E8FE1F47602C67)
1. [Adaptado de Rowel Atienza, Advance Deep Learning with Tensorflow 2 and Keras,Pack,2020](https://www.amazon.com/-/es/Rowel-Atienza-ebook/dp/B0851D5YQQ).
1. [Sutton, R. S., & Barto, A. G. (2018).Reinforcement learning: An introductio, MIT Press, 2018](https://web.stanford.edu/class/psych209/Readings/SuttonBartoIPRLBook2ndEd.pdf)
1. [Ejecutar en Colab](https://colab.research.google.com/drive/1ExE__T9e2dMDKbxrJfgp8jP0So8umC-A#sandboxMode=true&scrollTo=2XelFhSJGWGX)
1. [Human-level control through deep reinforcement
learning](https://web.stanford.edu/class/psych209/Readings/MnihEtAlHassibis15NatureControlDeepRL.pdf) 

## <span style="color:blue">Contenido</span>

* [Introducción](#Introducción)
* [Gradientes de política](#Gradientes-de-política)
* [Algoritmo Refuerzo (Reinforce)](#Algoritmo-Refuerzo-(Reinforce)
* [Ejemplo CartPole](#Ejemplo-CartPole )
* [Refuerzo con línea base](#Refuerzo-con-línea-base)


## <span style="color:blue">Introducción</span>

En las lecciones previas hemos hablado de la política de un agente, pero no hemos desarrollado ningún procedimiento que directamente la incluya como parte central de los algoritmos.

Por otro lado, hasta el momento hemos trabajado con modelos que tienen pocas acciones posibles, por lo que el método de entropía cruzada y el método Q-learning y el método DQN, los cuales están basados en redes neuronales de clasificación aplican bien al tipo de problemas abordado. Pero, ¿Qué sucede si el número de posibles acciones es muy grande o incluso infinito o no contable?

Piense por un momento en el problema de conducción automática. Un acción posible es girar el timón para cambiar ligeramente la dirección para por ejemplo evitar un obstáculo. El ángulo de giro que corresponde a la respectiva acción es un número, posiblemente un número real.


De otro lado, nos hemos enfocado en los valores de los estados o de las acciones dados los estados (Q-valores). Estos son números que aunque están vinculados con la política, no son probabilidades. 

En la practica, cambios ligeros en el proceso de optimización de las funciones de pérdida pueden ocasionar cambios importantes en estos valores. Por otro lado, si la salida de la red neuronal es un vector de probabilidades, tales cambios por lo general resultan más suaves. Por todas razones, en esta lección pondremos nuestro objetivo en la política del agente.

### La hipótesis de la recompensa

El aprendizaje reforzado se basa en que toda las metas y propósitos de una agente pueden ser pensados en términos de la optimización del valor esperado de la suma acumulativa de una señal escalar recibida que llamamos recompensa.

### Procesos de decisión de Markov

El agente trabaja siguiendo un proceso de decisión de Markov (PDM). el cual consiste de una decisión (¿Cuál acción tomar?) que debe tomar en cada estado. Esto da origen a una sucesión de estados, acciones y recompensas llamada  `trayectoria` y que se puede visualizar como

$$
s_0, a_0, r_1, s_1, a_1,r_2,\cdots
$$

y el objetivo es maximizar este conjunto de recompensas.

Un `Proceso de decisión de Markov (PDM)` es una tupla $(S,A,R,p, \gamma)$ tal que

$$
p(s',r| s,a) = Pr[s_{t+1}= s', R_{t+1}=r|S_t=s, A_t=a]
$$

$$
G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2R_{t+3} + \cdots
$$

en donde $S_t \in S$ (espacio de estados), $A_t \in A$(espacio de acciones), $R_t \in R$ (espacio de recompensas). $p$ se denomina dinámica del ambiente. 

En palabras simples un PDM define la probabilidad de transición a un nuevo estado $s'$ recibir una recompensa $r$ partiendo del estado actual $s$ y ejecutando una acción $a$.

Técnicamente es un proceso de Markov de primer orden. Un elemento importante del modelo es el factor de descuento $\gamma$, el cual es un valor entre 0 y 1. Sumando las recompensas futuras a lo largo del tiempo, descontadas con una potencia del factor de descuento se obtiene el concepto de retorno $G_t$. 

La idea central es que el agente encuentre trayectorias que maximizen el valor esperado del retorno.

La dinámica del ambiente $p$ está fuera del alcance del agente. Recuerde el juego de Frozen Lake. Un ejemplo de la vida real puede ser el siguiente. Suponga que se encuentra en un lugar con demasiado viento y conduce un vehículo muy liviano. O puede imaginarse en un velero. Se puede intentar ir en una determinada dirección (la acción). Pero el viento extremo lo puede conducir en otra dirección. Si embargo puede ser que sea posible elegir una dirección diferente que permita ir en la dirección correcta. Esta es la política, que el agente si controla.
Política

### Política

Cuando un agente sigue una política $\pi$, genera una sucesión de estados, acciones y recompensas que denominaremos trayectoria. Técnicamente la  política se define la probabilidad de acciones dado un estado:

$$
\pi(A_t=a|S_t=s), \hspace{3mm} \forall A_t\in A(s), S_t\in S.
$$

## <span style="color:blue">Gradientes de política</span>

El `objetivo del aprendizaje reforzado` (AR) es maximizar la recompensa $r$ cuando sigue un política parametrizada $J$:

$$
J(\theta)= \mathbb{E}_{\pi}[r(\tau)],
$$

en donde $r(\tau)$ representa la recompensa total para una trayectoria $\tau$.

Puede demostrarse que bajos ciertos supuestos que generalmente se tienen en AR todo PDM finito tiene al menos una política optima en el sentido de la recompensa obtneida y que entre todas las políticas optimales al menos una es estacionaria y determinista.

El procedimiento clásico para estimar el parámetro $\theta$ es el método del gradiente descendiente, el cual ( en términos muy simples) se basa en la regla de actualización

$$
\theta_{t+1} = \theta_t + \alpha \nabla J(\theta_t).
$$

$\nabla J(\theta_t)$ es el gradiente de la política y $\alpha$ una rata de aprendizaje.

El reto que tenemos es encontrar el gradiente de la política. El primer  problema es que $J(\theta)$ está definido como una esperanza (¡una integral!). 

Bajo algunas condiciones relacionadas con las derivadas de $\pi$  es posible pasar el gradiente a través de la integral como sigue


$$
\begin{align}
\nabla \mathbb{E}_{\pi}[r(\tau)] &= \nabla \int \pi(\tau)r(\tau)d\tau\\
& =   \int\nabla \pi(\tau)r(\tau)d\tau\\
& =   \int \pi(\tau)\nabla \log\pi(\tau)r(\tau)d\tau\\
\end{align}
$$

Por lo que se tiene el siguiente resultado

#### Teorema del gradiente de la política

$$
\nabla \mathbb{E}_{\pi_{\theta}}[r(\tau)] = \mathbb{E}_{\pi_{\theta}}[r(\tau)\nabla \log \pi_{\theta}(\tau)]
$$



Si se expande la definición de $\pi_{\theta}(\tau)$ se obtiene

$$
\pi_{\theta}(\tau) = P(s_0)\prod_{t=1}^{T}\pi_{\theta}(a_t|s_t)p(s_{t+1}, r_{t+1}|s_t, a_t).
$$

$P$ representa la distribución del estado inicial $s_0$ y se ha aplicado la regla del producto de la probabilidad y el hecho de ser un proceso de Markov que implica que cada nueva acción es independiente de la anterior. $T$ representa la longitud de la trayectoria. 

Tomando logaritmo se obtiene que

$$
\log \pi_{\theta}(\tau) = \log P(s_0) + \sum_{t=1}^{T}\log \pi_{\theta}(a_t|s_t) + \sum_{t=1}^{T} \log p(s_{t+1}, r_{t+1}|s_t, a_t).
$$

Por lo tanto se tiene que

$$
\nabla J(\theta) = \nabla \mathbb{E}_{\pi_{\theta}}[r(\tau)] = \mathbb{E}_{\pi_{\theta}}\left[ r(\tau) \left(\sum_{t=1}^{T}\nabla \log \pi_{\theta}(a_t|s_t)\right) \right]
$$

Este resultado dice que no es necesario conocer la distribución del estado inicial $P$ ni la dinámica del ambiente $p$ para calcular el gradiente de la política.

Los algoritmos que usan este resultado son conocidos como `algoritmos libres del modelo`, debido a que no modelamos el ambiente.

### MCMC

En la última ecuación se tiene que el cálculo del gradiente involucra una esperanza, es decir una integral, la cual por lo general es intratable. Aquí es ne donde entran en acción las técnicas Monte Carlo Markov Chain (MCMC). La idea central es que en cada paso de tiempo (cada iteración) se hace lo siguiente:

1. Se obtiene una muestra aleatoria grande de acciones siguiendo la distribución (política) $\pi(\theta^*)$, en donde $\theta^*$ es la estimación actual del parámetro $\theta$.
1. Se calcula el promedio de la expresión del gradiente para la muestra obtenida.


### Recompensa de la trayectoria

El termino $r(\tau)$ de la recompensa total de la trayectoria $\tau$ permanece inmutable en la expresión de $\nabla J(\theta)$. Este gradiente parametrizado no depende de  $r(\tau)$. Sin embargo el termino agrega bastante varianza en el muestreo MCMC. En realidad hay $T$ fuentes de variación debidas a  cada $R_t$. En su lugar podemos hacer uso del retorno $G_t$ debido a que desde el punto de vista de la  optimización del objetivo de AR, la recompensa pasada no contribuye para nada.

Cuando se reemplaza $r(\tau)$ por el retorno descontado $G_t$ arribamos al algoritmo clásico del gradiente de la política llamado reforzamiento (`reinforce`)

## <span style="color:blue">Método Refuerzo (Reinforce) </span>

El algoritmo reinforce se basa en aproximar el gradiente de la política como 


$$
\nabla J(\theta) = \nabla \mathbb{E}_{\pi_{\theta}}[r(\tau)] = \mathbb{E}_{\pi_{\theta}}\left[  \sum_{t=1}^{T}G_t \nabla\log \pi_{\theta}(a_t|s_t) \right]
$$


En realidad, en la formula anterior no se ha resuelto el problema de la varianza en las trayectorias muestreadas. Desde el punto de vista de la inferencia bayesiana, se sabe que cuando el tamaño de muestra es grande, no importa la a priori seleccionada. Esto implica que con muestras grandes de trayectorias la distribución del estado inicial no es importante, el algortimo MCMC converge al modelo de los parámetros verdaderos. El problema es que varianzas muy grandes el problema de estabilizar los parámetros del modelo es bastante difícil. Resolveremos este problema en la lección Actor-crítico.

De momento examinemos la implementación usual del método reinforce.

Como hemos hecho antes a lo largo del curso, vamos a usar la aproximación del retorno con la función de Q-valores $Q(s,a)$. Así, para cada paso de la trayectoria tenemos que

$$
\nabla J(\theta) \approx \mathbb{E}_{\pi_{\theta}}\left[  Q(s,a) \nabla\log \pi_{\theta}(a|s) \right]
$$



### Interpretación

El gradiente de la política indica la dirección en la cual se deben cambiar los parámetros de la red neuronal para mejorar la política en términos de la recompensa total acumulada. En esta aproximación el gradiente se escala proporcionalmente al valor de la acción tomada $Q(s,a)$ y el gradiente en sí mismo es igual al gradiente del logaritmo de la probabilidad de la acción tomada. 

Esto significa que estamos tratando de aumentar la probabilidad de acciones que nos han dado buena recompensa total y disminuyen la probabilidad de acciones con
malos resultados finales. 

La esperanza es aproximada mediante el promedio del gradiente en varios pasos, de acuerdo con las técnicas MCMC.


### Función de pérdida y máxima log-verosimilitud

La función de pérdida que usualmente se utiliza en los métodos de gradiente de la política es

$$
\mathfrak{L} = -Q(s,a) \log\pi(a|s)
$$

Es función de pérdida es menos la log-verosimilitud del modelo estadístico

$$
L = \pi(a|s)^{Q(s,a)}.
$$

Por ejemplo para una trayectoria $\tau$ con $T$ pasos se tiene que

$$
\log L(\tau) = \sum_{t=1}^{T} Q(s_t,a_t)\log\pi(a_t|s_t)
$$

### El algortimo Reinforce

1. Inicialice la red neuronal con pesos aleatorios.
1. Corra N episodios completos (una trayectoria), almacenando  las transiciones $(s,a,r,s')$.
1. Para cada paso, $t$, de cada uno de los episodios,  $k$, calcule la recompensa total descontada para la sucesión de pasos: $Q_{t,k}= \sum_{i=0} \gamma^i r_i$.
1. Calcule la función de pérdida para todas las transiciones: $\mathfrak{L} = - \sum_{tk} Q_{t,k} \log(\pi(s_{t,k},a_{t,k}))$
1. Ejecute el paso de actualización de pesos del algoritmo SGD seleccionado.
1. Repita desde el paso 2 hasta convergencia.

### Diferencias con Q-learning

El método reinforce se diferencia del método Q-learning en varias cosas.

1. No se requiere una exploración explícita de tipo epsilon-greedy, que se usa en Q-learning para evitar que el algoritmo quede estancado en un mínimo local. Ahora la red neuronal retorna probabilidades directamente. Al comienzo la red es inicializada con valores aleatorios, por lo que la primera salida corresponde a una distribución uniforme.
1. No se usa la memoria de repetición (replay buffer). Los métodos gradiente de política son métodos basados en la política (Q-learning es libre de política). Es decir, los datos obtenidos para el entrenamiento so se basan en políticas viejas. Esto es bueno y malo. Lo  bueno es que estos métodos convergen más rápido por lo general. Lo malo es que  requieren por lo general mucha más interacción con el ambiente que por ejemplo DQN.
1. No se requiere una red neuronal objetivo (target). Aquí se usan los Q-valores, pero ya no son aproximados como DQN, sino que se calculan completamente en cada trayectoria. En DQN se requiere la red target para romper la correlación entre valores $Q(s,a)$, porque en cada paso ellos son aproximados. Aquí los Q-valores se calculan sin aproximación.

## <span style="color:blue"> Ejemplo CartPole </span>

En esta sección vemos el método en acción. usaremos el problema CartPole para ilustrar el método con código real.


### Importa  módulos

In [6]:
#!/usr/bin/env python3
import gym
#import ptan
import numpy as np
#from torch.utils.tensorboard import SummaryWriter

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#ptan
import actions
import agent
import experience

import utils

### Hiperparámetros generales

In [2]:
GAMMA = 0.99
LEARNING_RATE = 0.01
EPISODES_TO_TRAIN = 4

### Clase PGN (Policy Gradient Network)

Esta es la red neuronal que usaremos. Ya familiar para todos.
Observe que la red no regresa probabilidades como hemos previsto. Esto se hace porque en la implementación decidimos usar en la perdida la función *log_softmax* que calcula de manera eficiente el logaritmo del softmax que ne realidad es lo que se requiere para la función de pérdida. Además este cálculo es bastante más estable que calcular inicialmente softmax y luego el logaritmo.

In [3]:
class PGN(nn.Module):
    def __init__(self, input_size, n_actions):
        super(PGN, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, n_actions)
        )

    def forward(self, x):
        return self.net(x)

### Cálculo de los Q-valores

Con la siguiente función se hace el cálculo de los Q-valores de manera eficiente.

In [4]:
def calc_qvals(rewards):
    res = []
    sum_r = 0.0
    for r in reversed(rewards):
        sum_r *= GAMMA
        sum_r += r
        res.append(sum_r)
    return list(reversed(res))

### Entrenamiento

In [5]:
if __name__ == "__main__":
    env = gym.make("CartPole-v0")
    #writer = SummaryWriter(comment="-cartpole-reinforce")

    net = PGN(env.observation_space.shape[0], env.action_space.n)
    print(net)

    agente = agent.PolicyAgent(net, preprocessor= agent.float32_preprocessor,
                                   apply_softmax=True)
    exp_source = experience.ExperienceSourceFirstLast(env, agente, gamma=GAMMA) # iterator

    optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)

    total_rewards = []
    step_idx = 0
    done_episodes = 0

    batch_episodes = 0
    batch_states, batch_actions, batch_qvals = [], [], []
    cur_rewards = []

    for step_idx, exp in enumerate(exp_source):
        batch_states.append(exp.state)
        batch_actions.append(int(exp.action))
        cur_rewards.append(exp.reward)

        if exp.last_state is None:
            batch_qvals.extend(calc_qvals(cur_rewards))
            cur_rewards.clear()
            batch_episodes += 1

        # handle new rewards
        new_rewards = exp_source.pop_total_rewards()
        if new_rewards:
            done_episodes += 1
            reward = new_rewards[0]
            total_rewards.append(reward)
            mean_rewards = float(np.mean(total_rewards[-100:]))
            print("%d: reward: %6.2f, mean_100: %6.2f, episodes: %d" % (
                step_idx, reward, mean_rewards, done_episodes))
            #writer.add_scalar("reward", reward, step_idx)
            #writer.add_scalar("reward_100", mean_rewards, step_idx)
            #writer.add_scalar("episodes", done_episodes, step_idx)
            if mean_rewards > 195:
                print("Solved in %d steps and %d episodes!" % (step_idx, done_episodes))
                break

        if batch_episodes < EPISODES_TO_TRAIN:
            continue

        optimizer.zero_grad()
        states_v = torch.FloatTensor(batch_states)
        batch_actions_t = torch.LongTensor(batch_actions)
        batch_qvals_v = torch.FloatTensor(batch_qvals)

        logits_v = net(states_v)
        log_prob_v = F.log_softmax(logits_v, dim=1)
        log_prob_actions_v = batch_qvals_v * log_prob_v[range(len(batch_states)), batch_actions_t]
        loss_v = -log_prob_actions_v.mean()

        loss_v.backward()
        optimizer.step()

        batch_episodes = 0
        batch_states.clear()
        batch_actions.clear()
        batch_qvals.clear()

    #writer.close()

PGN(
  (net): Sequential(
    (0): Linear(in_features=4, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=2, bias=True)
  )
)
11: reward:  11.00, mean_100:  11.00, episodes: 1
23: reward:  12.00, mean_100:  11.50, episodes: 2
37: reward:  14.00, mean_100:  12.33, episodes: 3
48: reward:  11.00, mean_100:  12.00, episodes: 4
65: reward:  17.00, mean_100:  13.00, episodes: 5
75: reward:  10.00, mean_100:  12.50, episodes: 6
87: reward:  12.00, mean_100:  12.43, episodes: 7
100: reward:  13.00, mean_100:  12.50, episodes: 8
111: reward:  11.00, mean_100:  12.33, episodes: 9
126: reward:  15.00, mean_100:  12.60, episodes: 10
137: reward:  11.00, mean_100:  12.45, episodes: 11
158: reward:  21.00, mean_100:  13.17, episodes: 12
167: reward:   9.00, mean_100:  12.85, episodes: 13
209: reward:  42.00, mean_100:  14.93, episodes: 14
219: reward:  10.00, mean_100:  14.60, episodes: 15
237: reward:  18.00, mean_100:  14.81, episodes: 16
257: reward:  20

### Comparación con DQN

En la siguiente imagen tomada del texto del Maxi Lapan, página 292, se compara en condicones similares el comportamiento d elos algoritmos DQN i REINFORCE. Los datos por supuesto dependen de los valres pseudo aleatorios inicilaes, pero claramente el problema es resuleto mucho más rápido con REINFORCE. Esto se conoce en la literatura y ha sido  la motivación de estudiar el método. En la siguiente sección hacemos una ampliación en la cual se trata de resolver el problema de la varianza introducida en el gradiente de política por el retorno $G_t$.


<figure>
<img src="../Imagenes/DQN_Reinforce.png"  width="400" height="400" align="center"/> 
</figure>

Fuente: Maxi Lapan

### Métodos de política v.s. Métodos basados en valores

Los métodos de política son una generalización del método de entropía cruzada.

+ Los métodos de política optimizan directamente la política. Los  métodos de valor, como DQN, hacen lo mismo indirectamente.
+ Los métodos de política están basados totalmente en la política y requieren muestras frescas del ambiente. Los métodos de valor pueden beneficiarse de datos antiguos, obtenidos de la vieja política y otras fuentes.
+ Los métodos de política suelen ser menos eficientes en la muestra, lo que significa que requieren mayor interacción con el entorno. Los métodos de valor pueden beneficiarse de grandes búferes de reproducción (replay buffer). Sin embargo, la eficiencia de la muestra no significa que el valor Los métodos son más eficientes desde el punto de vista computacional y, muy a menudo, es todo lo contrario.
+ En el ejemplo anterior, durante el entrenamiento, necesitábamos acceder a nuestro RN una sola vez, para obtener las probabilidades de las acciones. En DQN, necesitamos procesar dos lotes de estados: uno para el estado actual y otro para el estado siguiente en la actualización de Bellman.

En general hay situaciones en donde DQN es más natural. >Por ejmplo, las soluiciones del estado del arte para juegos de Atari son variciones de DQN. En constraste los métodos de política son mejor aplicados en problemas de control contínuo o en casos en donde el acceso al ambiente de barato y rápido.

## <span style="color:blue">Refuerzo con línea base </span>

Ya mencionamos antes que nuestra aproximación del gradiente de política dada por $\nabla J \approx \mathbb{E}[Q(s,a)\nabla \pi(s|a)]$ es proporcional a la recompensa descontada desde un estado dado. Esta recompensa depende por supeuesto del ambiente e introduce una fuente importante de variación. Una solución a este incoveniente substraer de la recompensa una contante. Posibles solucioness son restar de la recompensa
 
+ La media de las recompensas descontadas.
+ La media móvil de las recompensas descontadas.
+ El valor del estado V(s).

La última solución es introducida en la lección actor-crítico.

Por otro lado para recortar los episodios y no hace cálculos innecesarios se puede determinar a parir de que momento el factor de decuento es tan pequeño que más pasos en el episodio no aportan mucho al cáculo de la recompensa dscontada. Por ejemplo, $0.9^{50}= 0.005$. Entonces si el factor de descuento es $\gamma=0.9$, quizás sea suficiente parar después de 50 pasos en un episodio.


La introducción de un constante que reste a la recompensa descontada se basa en el siguiente hecho.

Puede verificarse que

$$
\mathbb{E}_{\theta}\left[\left( \nabla \log \pi_{\theta}(a_t|s_t) \right) \right]=0.
$$

En efecto, bajo el supuesto que la integral y el gradiente pueden intercambiarse, se tiene que

$$
\mathbb{E}_{\theta}\left[\left( \nabla \log \pi_{\theta}(a|s) \right) \right] =\int \pi_{\theta}(a|s)\nabla \log \pi_{\theta}(a|s)d\tau =  \int \nabla \pi_{\theta}(a|s)d\tau= \nabla\int  \pi_{\theta}(a|s)d\tau = \nabla  1 = 0.
$$

La verificación meticulosa, se  deja como ejercicio al lector interesado. Sea $b$ una variable que no depende de la trayectoria $\tau$.  Esta variable se denominará `línea base`. Debido al resultado anterior se tiene que

$$
\nabla J(\theta) = \nabla \mathbb{E}_{\pi_{\theta}}[r(\tau)] = \mathbb{E}_{\pi_{\theta}}\left[  \sum_{t=1}^{T}(G_t-b)\nabla\log \pi_{\theta}(a_t|s_t) \right].
$$

Usando $b$ en teoría y en la práctica la varianza puede ser reducida, manteniendo el gradiente de la política insesgado. Un bune valor que puede usarse como línea base el valor del estado actual.

### Ejemplo de implementación de Gradiente de política con línea base

In [ ]:
#!/usr/bin/env python3
import gym
import ptan
import numpy as np
#from torch.utils.tensorboard import SummaryWriter
from typing import Optional

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
GAMMA = 0.99
LEARNING_RATE = 0.001
ENTROPY_BETA = 0.01
BATCH_SIZE = 8

REWARD_STEPS = 10

In [ ]:
class PGN(nn.Module):
    def __init__(self, input_size, n_actions):
        super(PGN, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, n_actions)
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
def smooth(old: Optional[float], val: float, alpha: float = 0.95) -> float:
    if old is None:
        return val
    return old * alpha + (1-alpha)*val

In [ ]:
if __name__ == "__main__":
    env = gym.make("CartPole-v0")
    #writer = SummaryWriter(comment="-cartpole-pg")

    net = PGN(env.observation_space.shape[0], env.action_space.n)
    print(net)

    agent = ptan.agent.PolicyAgent(net, preprocessor=ptan.agent.float32_preprocessor,
                                   apply_softmax=True)
    exp_source = ptan.experience.ExperienceSourceFirstLast(
        env, agent, gamma=GAMMA, steps_count=REWARD_STEPS)

    optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)

    total_rewards = []
    step_rewards = []
    step_idx = 0
    done_episodes = 0
    reward_sum = 0.0
    bs_smoothed = entropy = l_entropy = l_policy = l_total = None

    batch_states, batch_actions, batch_scales = [], [], []

    for step_idx, exp in enumerate(exp_source):
        reward_sum += exp.reward
        baseline = reward_sum / (step_idx + 1)
        #writer.add_scalar("baseline", baseline, step_idx)
        batch_states.append(exp.state)
        batch_actions.append(int(exp.action))
        batch_scales.append(exp.reward - baseline)

        # handle new rewards
        new_rewards = exp_source.pop_total_rewards()
        if new_rewards:
            done_episodes += 1
            reward = new_rewards[0]
            total_rewards.append(reward)
            mean_rewards = float(np.mean(total_rewards[-100:]))
            print("%d: reward: %6.2f, mean_100: %6.2f, episodes: %d" % (
                step_idx, reward, mean_rewards, done_episodes))
            #writer.add_scalar("reward", reward, step_idx)
            #writer.add_scalar("reward_100", mean_rewards, step_idx)
            #writer.add_scalar("episodes", done_episodes, step_idx)
            if mean_rewards > 195:
                print("Solved in %d steps and %d episodes!" % (step_idx, done_episodes))
                break

        if len(batch_states) < BATCH_SIZE:
            continue

        states_v = torch.FloatTensor(batch_states)
        batch_actions_t = torch.LongTensor(batch_actions)
        batch_scale_v = torch.FloatTensor(batch_scales)

        optimizer.zero_grad()
        logits_v = net(states_v)
        log_prob_v = F.log_softmax(logits_v, dim=1)
        log_prob_actions_v = batch_scale_v * log_prob_v[range(BATCH_SIZE), batch_actions_t]
        loss_policy_v = -log_prob_actions_v.mean()

        prob_v = F.softmax(logits_v, dim=1)
        entropy_v = -(prob_v * log_prob_v).sum(dim=1).mean()
        entropy_loss_v = -ENTROPY_BETA * entropy_v
        loss_v = loss_policy_v + entropy_loss_v

        loss_v.backward()
        optimizer.step()

        # calc KL-div
        new_logits_v = net(states_v)
        new_prob_v = F.softmax(new_logits_v, dim=1)
        kl_div_v = -((new_prob_v / prob_v).log() * prob_v).sum(dim=1).mean()
        #writer.add_scalar("kl", kl_div_v.item(), step_idx)

        grad_max = 0.0
        grad_means = 0.0
        grad_count = 0
        for p in net.parameters():
            grad_max = max(grad_max, p.grad.abs().max().item())
            grad_means += (p.grad ** 2).mean().sqrt().item()
            grad_count += 1

        bs_smoothed = smooth(bs_smoothed, np.mean(batch_scales))
        entropy = smooth(entropy, entropy_v.item())
        l_entropy = smooth(l_entropy, entropy_loss_v.item())
        l_policy = smooth(l_policy, loss_policy_v.item())
        l_total = smooth(l_total, loss_v.item())
        """
        writer.add_scalar("baseline", baseline, step_idx)
        writer.add_scalar("entropy", entropy, step_idx)
        writer.add_scalar("loss_entropy", l_entropy, step_idx)
        writer.add_scalar("loss_policy", l_policy, step_idx)
        writer.add_scalar("loss_total", l_total, step_idx)
        writer.add_scalar("grad_l2", grad_means / grad_count, step_idx)
        writer.add_scalar("grad_max", grad_max, step_idx)
        writer.add_scalar("batch_scales", bs_smoothed, step_idx)

        """
        
        batch_states.clear()
        batch_actions.clear()
        batch_scales.clear()

    #writer.close()